In [2]:
import pandas as pd

df = pd.read_parquet("/Users/muaazshaikh/aadhar-forecast-anomaly/data/raw_combined.parquet")
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1439021 entries, 0 to 1439020
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   date           1439021 non-null  datetime64[us]
 1   state          1439021 non-null  category      
 2   district       1439021 non-null  category      
 3   pincode        1439021 non-null  string        
 4   demo_age_5_17  1439021 non-null  Int32         
 5   demo_age_17_   1439021 non-null  Int32         
dtypes: Int32(2), category(2), datetime64[us](1), string(1)
memory usage: 48.0 MB


In [4]:
df.shape

(1439021, 6)

In [5]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,2025-01-03,uttar pradesh,gorakhpur,273213,49,529
1,2025-01-03,andhra pradesh,chittoor,517132,22,375
2,2025-01-03,gujarat,rajkot,360006,65,765
3,2025-01-03,andhra pradesh,srikakulam,532484,24,314
4,2025-01-03,rajasthan,udaipur,313801,45,785


In [6]:
df.tail()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
1439016,2025-12-29,west bengal,west midnapore,721212,0,12
1439017,2025-12-29,west bengal,west midnapore,721420,0,1
1439018,2025-12-29,west bengal,west midnapore,721424,0,5
1439019,2025-12-29,west bengal,west midnapore,721426,0,3
1439020,2025-12-29,west bengal,hooghly,712701,0,1


In [7]:
df.isna().mean().sort_values(ascending=False)

date             0.0
state            0.0
district         0.0
pincode          0.0
demo_age_5_17    0.0
demo_age_17_     0.0
dtype: float64

In [8]:
df.duplicated().sum()

np.int64(0)

In [10]:
df.duplicated(subset=["date", "state", "district", "pincode"]).sum()

np.int64(21152)

In [12]:
df["date"].min(), df["date"].max()

(Timestamp('2025-01-03 00:00:00'), Timestamp('2025-12-29 00:00:00'))

In [13]:
sample_districts = (
    df[["state", "district"]].drop_duplicates().sample(20, random_state=42)
)

def date_gap_check(sub):
    dates = sub.sort_values("date")["date"]
    return dates.diff().dt.days.max()

gap_report = (
    df.merge(sample_districts, on=["state", "district"]).groupby(["state", "district"]).apply(date_gap_check)
)

gap_report.sort_values(ascending=False)

state           district          
telangana       medchal?malkajgiri    241.0
west bengal     south dumdum(m)       129.0
meghalaya       south garo hills       58.0
chhattisgarh    balrampur              31.0
jharkhand       chatra                 31.0
telangana       jangaon                29.0
odisha          angul                  29.0
nagaland        mon                    29.0
karnataka       vijayapura             29.0
                bagalkot               29.0
maharashtra     buldhana               28.0
madhya pradesh  neemuch                28.0
bihar           nalanda                28.0
madhya pradesh  barwani                28.0
rajasthan       sawai madhopur         28.0
telangana       jogulamba gadwal       28.0
                kamareddy              28.0
uttar pradesh   lucknow                28.0
west bengal     bankura                28.0
andhra pradesh  karimnagar             28.0
dtype: float64

In [14]:
df[["demo_age_5_17", "demo_age_17_"]].describe()

,demo_age_5_17,demo_age_17_
count,1439021.0,1439021.0
mean,2.607734,23.701038
std,16.682467,139.647265
min,0.0,0.0
25%,0.0,2.0
50%,1.0,6.0
75%,2.0,15.0
max,2690.0,16166.0


In [16]:
(df[["demo_age_5_17", "demo_age_17_"]] < 0).sum()

demo_age_5_17    0
demo_age_17_     0
dtype: Int64

In [17]:
(df["demo_age_5_17"] == 0).mean(), (df["demo_age_17_"] == 0).mean()

(np.float64(0.4746525589272151), np.float64(0.019504927308218574))

In [18]:
df["state"].nunique()

58

In [19]:
(
    df.groupby("district")["state"]
      .nunique()
      .sort_values(ascending=False)
      .head(10)
)

district
hooghly     5
daman       3
diu         3
kargil      3
howrah      3
baudh       2
bargarh     2
karaikal    2
debagarh    2
baramula    2
Name: state, dtype: int64

In [20]:
df.assign(
    total=df["demo_age_5_17"].fillna(0) + df["demo_age_17_"].fillna(0)
).groupby(["state", "district"])["total"].mean().sort_values(ascending=False).head(10)


state          district        
delhi          north east delhi    432.463415
               west delhi          287.266281
               east delhi          267.758437
               north west delhi    242.459046
chhattisgarh   mahasamund          226.059211
bihar          kishanganj          223.719921
delhi          north delhi         220.734317
chhattisgarh   narayanpur          206.628571
uttar pradesh  pilibhit            187.056122
               rampur              176.998403
Name: total, dtype: Float64